# Notebook 02: GNN-Transformer Training

## The Solution: Physics-Informed Deep Learning

**Learning Objective:** Train a GNN-Transformer model and see smooth, physics-compliant predictions!

### Architecture

```
Graph → GNN → Isotope Embeddings → Transformer → Smooth σ(E)
```

This combines:
1. **GNN**: Learns nuclear topology (which isotopes are related)
2. **Transformer**: Learns smooth energy sequences (no staircase!)

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt

from nucml_next.data import NucmlDataset
from nucml_next.model import GNNTransformerEvaluator, GNNTransformerTrainer
from nucml_next.physics import PhysicsInformedLoss

print("✓ Imports successful")

### Step 1: Initialize Model

In [ ]:
# Create dataset
dataset = NucmlDataset(mode='graph')

# Initialize GNN-Transformer
model = GNNTransformerEvaluator(
    node_features=7,
    gnn_embedding_dim=32,
    gnn_num_layers=3,
    transformer_num_layers=4,
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

### Step 2: Train with Physics-Informed Loss

In [ ]:
# Prepare training data
trainer = GNNTransformerTrainer(model)
train_data = trainer.prepare_training_data(dataset)

# Train
history = model.train_model(
    train_data[:50],  # Use subset for demo
    epochs=20,
    learning_rate=1e-3,
)

# Plot training curves
model.plot_training_history(history)

### Step 3: Compare Predictions

In [ ]:
# Get predictions for U-235 capture
energies = np.logspace(0, 2, 500)  # 1-100 eV
isotope_idx = dataset.graph_builder.isotope_to_idx.get((92, 235))

# Predict
gnn_pred = model.predict_isotope(
    dataset.graph_builder.build_global_graph(),
    isotope_idx,
    energies
)

# Plot: GNN-Transformer produces SMOOTH curves!
plt.figure(figsize=(12, 6))
plt.plot(energies, gnn_pred, 'g-', lw=2.5, label='GNN-Transformer (Smooth!)')
plt.xlabel('Energy (eV)', fontsize=12, fontweight='bold')
plt.ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
plt.title('GNN-Transformer: Physics-Compliant Predictions', fontsize=14, fontweight='bold')
plt.legend()
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

print("\n✓ SUCCESS: No staircase effect!")
print("✓ Smooth resonance curves")
print("✓ Physics-compliant behavior")

### 🎓 Key Takeaway

> GNN-Transformer learns **smooth** predictions that respect physics!
>
> But are they **reactor-accurate**? → Notebook 03!

Continue to `03_OpenMC_Loop_&_Inference.ipynb` →